# Neural network supercointegrated pairs trading (Part 1)
## A machine learning augmented algorithmic trading strategy

Pairs trading is a common and relatively simple algorithmic trading technique that can act as a gentle introduction to quantitative finance. Its understanding does not require much, if at all stochastic calculus, but it requires some knowledge of statistics and timeseries analysis from econometrics. I learned a lot of this stuff by hobby on the side of my undergraduate degree, while the statistics I covered in my thesis for UNSW. In this article, I will demonstrate one potential workflow with the following steps.

1. Data acquisiton, using the IEX Cloud API (the Sandbox tool is free provided one creates an account).
2. Data preprocessing, including finding our supercointegrated pairs and splitting the data into training and test sets.
3. Train and implement the model.
4. Backtest using PyFolio.
5. Optimise using any number options - I used a genetic algorithm. 

On writing this article I ran into a few problems. Minutely data would usually be the minimum requirement for a quantitative finance application, but for long term free data for the purpose of this article I stuck with the relatively easy to use IEX Cloud API, which offers unlimited free Sandbox data (simulated "fake" testing data). Learning how to implement and interpret the cointegration tests to find the supercointegrated pairs according to <cite data-cite="6461367/F7Z9Z9M5"></cite>. Implementing them manually as in this [Medium post](https://medium.com/bluekiri/cointegration-tests-on-time-series-88702ea9c492) can educational before using the optimised robust version provided by Python's ```statsmodels``` library. 

First I will be briefly outlining the theory behind cointegration and pairs trading, followed by the LTSM neural network model I used, and why it was used. This is all available on Wikipedia, and it is explained better than I can in many places, so I will outline and direct readers toward more comprehensive sources where I can. I have accessed some academic papers, and largely they are freely readable in their respective journals. After the theory, I will show how each component of this model was implemented in Python 3 along with the results and observations made along the way to help with the analysis. Finally, the end goal will be the backtested result of the model. In the next article I will demonstrate the genetic algorithm optimisation method that I enjoy using. 

The first step is to set our environment variable ```IEX_TOKEN``` to the token we get for making our free IEX Cloud API token. If you want to access the Sandbox data (recommended), follow these [steps](https://intercom.help/iexcloud/en/articles/2915433-testing-with-the-iex-cloud-sandbox). I've stored my ```IEX_TOKEN``` under an aptly named ```IEX_TEST_TOKEN``` environment variable in a windows environment. This might be slightly different for UNIX users, feel free to hardcode it in if no one else is going to see it.

In [1]:
import os
os.environ['IEX_TOKEN'] = os.environ['IEX_TEST_TOKEN']

The ```iexfinance``` API conveniently takes ```datetime``` variables. So we'll use the ```datetime``` module to set our start and end dates. 

In [2]:
from datetime import datetime
start = datetime(2017, 1, 1)
end = datetime.today()

Now webscrape Wikipedia for our our S&P 100 companies. I won't go into detail here.
```python

from bs4 import BeautifulSoup
import pandas as pd
import requests

def snp100_download_symbols():
    url = 'https://en.wikipedia.org/wiki/S%26P_100'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    tickers = []
    for row in soup.find('table', {'class': 'wikitable sortable'}).tbody.findAll('tr'):
        for col in row.findAll('td'):
            text = col.get_text().strip()
            tickers.append(text)
            break
    df = pd.DataFrame(index=range(len(tickers)))
    df['Symbols'] = tickers
    df.to_csv(open('data/snp100.csv', 'w'), index=False)

```

Now we're good to pull from IEX. This is one of the smoothest data pulls you'll have in your life. The ```time.sleep(0.2)``` method is here under the ```save_test_data``` function with ```iexfinance.stocks.get_historical_data()``` because the IEX API indicates that you will be throttled at a request rate with a lower period than 10ms (correct me if I'm wrong). I kept it at 200ms, but feel free to lower it if you don't want to wait 20 seconds or so. The following is from ```scripts.iex_timeseries.py```.

```python
from iexfinance.stocks import get_historical_data
import pickle
from time import sleep

def csv_fn(fund_type):
    return 'data/snp{}.csv'.format(fund_type)

def create_df(fund_type):
    return pd.read_csv(csv_fn(fund_type))

def symbols(fund_type):
    df = create_df(fund_type)    
    return df.Symbols.to_list()

def pickle_fn(fund_type):
    return 'data/snp{}_test_data.pickle'.format(fund_type)

def save_test_data(fund_type):
    snp_test_data = [
        get_historical_data(
            symbol, 
            start, 
            end, 
            output_format='pandas'
            ) \
        for symbol in symbols(fund_type) if sleep(0.2) is None
        ]
    pickle.dump(snp_test_data, open(pickle_fn(fund_type), 'wb'))

def load_test_data(fund_type):
    symbol_list = symbols(fund_type)
    data_load = pickle.load(open(pickle_fn(fund_type), 'rb'))
    return dict(zip(symbol_list, data_load))

```

I also have a webscrape for the S&P500 list of companies, hence the ```fund_type``` variable and the nested functions. A ```pickle``` is used here as it's going to be a 3D list of ```pandas``` dataframes and a CSV will not like that too much. Retrieving the serialised pickled data is quicker than saving the dataframe as a .xslx file or something like that. We could just save each symbol as a separate csv file, but I'd rather keep the number of files more compact for this project. Now we can grab our dataframe easily from our from our ```data``` directory with other scripts (or Notebooks) without running this whole script. And that's it for now for our data acquisition.

## Supercointegrated Pairs Theory 

### Vector Autoregression (VAR)

The sections covering vector autoregression and the vector error correction model will not be new to anyone familiar with ARMA or GARCH models, but ought to be briefly covered to explain the necessity for covariance testing later on. Additionally, VAR here is not to be confused with value at risk (VaR), which is not mentioned in this article, vector autoregression explains the evolution of a set of $k$ observations in a timeseries (called endogenous variables, handy later) with respect to a sample period $T$. In the notation used by the MIT OpenCourseWare [Applied Financial Mathematics Course](https://ocw.mit.edu/courses/mathematics/18-s096-topics-in-mathematics-with-applications-in-finance-fall-2013/), covariance stationary timeseris $X_t$ has an autoregressive representation of,
$$
    X_t = \big(\sum_{i=0}^{\infty}\psi_i^* X_{t-i}\big) + \eta_t,
$$
where the weighted lag operator $\psi(L)=\sum_{i=0}^{\infty}$ and $\eta_t$ represents linearly unpredictable white noise. Plainly put, the final term in the timeseries sequence may be represented as a linear projection on the preceding points, with some error term / white noise tacked onto it. This is common in stochastic processes and note that a covariance stationary timeseries is one that satisfies that $E(X_t)=\mu$, $Var(X_t)=\sigma_X^2$ and $Cov(X_t,X_{t+\tau})$ are all constant over time t. There are stationarity tests for this, mentioned later.  

Things get more complicated when the timeseries is non-stationary, i.e. orders of integration higher than $I(0)$. This is where one might use a vector error correction model.

NB: Endogeneous and exogeneous variables will come in handy later, when we implement the Augmented Dickey-Fuller Test (ADF).

### [Cointegration](https://en.wikipedia.org/wiki/Cointegration): A statistical property of a collection of timeseries variables
All of the series must be order *d* integrated. If two or more series are individually integrated of order *d* but some linear combination of them has a *lower order of integration* then they are said to be cointegrated. Mathematically, for $k$ distinct timeseries $\{X_i\}_{i=1}^k$ there exists a set of coefficients $\{A_i\}_{i=1}^k$, such that 
$$
    \sum_{i=1}^k A_i X_i 
$$
is less than *d* order integrated. Put plainly, the linear combination between a simple example of two $I(1)$ timeseries would be another timeseries of $I(0)$,
$$
    X_1 - \beta X_2 = I(0) 
$$
where $\beta$ is some scaling coefficient. This gives us a way of indicating whether there is a long-run stochastic relationship between two stochastic variables. For those visual learners, this is explained in this [video](https://www.youtube.com/watch?v=vvTKjm94Ars). In this case, the difference between these $I(1)$ timeseries with the help of our scaling coefficient $\beta$ results in some sort of random walk (which is $I(0)$).

### Vector error correction model (VECM)

Simple VAR models may require covariance stationary timeseries as inputs, but a test like the Johansen test doesn't require you to refactor your data into $I(0)$ stationary timeseries before testing. Although, inside it's operation it does just that. The reason for these models is when spurious correlation occurs between two timeseries. Spurious correlation is when two events are associated but not causally related. The association may come from an unseen factor, or coincidence. A VECM will break up a timeseries into a deterministic trend and a stationary series containing deviations to address this issue. Johansen begins with this in his 1991 paper <cite data-cite="6461367/V8Z5VVI4"></cite>.
$$
     \Delta X_t = \sum_{i=1}^{k-1}\Gamma_i \Delta X_{t-i} + \Pi X_{t-k} + \Phi D_t + \mu + \varepsilon_t
$$
In the case of Johansen's VECM model applied to a general VAR model as above, error correction features are added by first estimating the VAR with potentially integrated timeseries, and the cointegration is tested for iteratively. 

###  The Johansen Cointegration Test
Søren Johansen is a pretty sharp guy and he's still a permanent faculty member down at the University of Copenhagen, and it's not everyong that has a high-end statistical test as a canonical element of econometrics and statistics named after only them. The Johansen cointegration test gives cointegrated pairs within a selection of stock prices, as well as the hedge ratio which ought to be applied to each of the .A 1% significance level is based on the vector autogregressive model of order $\rho$, represented as,
$$
    y_t = \mu_t + A_1 \cdot y_{t-1} + A_2 \cdot y_{t-2} + 
    \cdots + A_\rho \cdot y_{t-\rho} + B \cdot x_t + 
    \epsilon_t
$$

The [Johansen](https://en.wikipedia.org/wiki/Johansen_test) Cointegration test, 
$$
    \Delta y_t = \mu_t + \Pi \cdot y_{t-1} + \sum_{i=1}^{\rho-1} \Gamma_i \cdot \Delta y_{t-i} + B \cdot x_t + \epsilon_t
$$
can be generalised to the case where $y(t)$ are vectors of multiple timeseries and incidently the cofficients $\Gamma$ and $\Pi$ are matrices.

From Johansen's original paper <cite data-cite="6461367/V8Z5VVI4"></cite>,
$$
    \Delta X_t = \sum_{i=1}^{k-1} \Delta X_{t-i} + \Pi X_{t-k} + \Phi D_t + \mu + \varepsilon_t
$$
- $D_t$: seasonal dummies (orthogonal to constant term)
- $\varepsilon_t$: $p$-dimensional Gaussian variables (zero-mean and variance matrix $\Lambda$)
- first $k$ data points $X_{1-k},\cdots,X_0$ are considered fixed and likelihood function is calculated for given values of these
- $\Gamma_1,\cdots,\Gamma_{k-1},\Phi,\mu$ and $\Lambda$ vary without restrictions
- hypothesis is formed based on restrictions on $\Pi$
- above is denoted as $H_1$

$$
    H_2: \Pi = \alpha \beta'
$$
where $\beta$, the cointegrating vectors, and $\alpha$ the adjustment coefficients are $p\times r$ matrices. Comparing models with different numbers of cointegrating vectors uses notation $H_2(r)$.

## Johansen Cointegration Test Implementation

In [3]:
# Some imports to make tables and graphs interactive and pretty.
import chart_studio
import chart_studio.plotly as py
import os
import pandas as pd
import plotly.figure_factory as ff
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "vscode"
chart_studio.tools.set_credentials_file(
    username=os.environ['PLOTLY_USERNAME'], 
    api_key=os.environ['PLOTLY_API_KEY']
)


A training dataset is used to calculate the supercointegrated pairs based on <cite data-cite="6461367/F7Z9Z9M5"></cite>, which also asserts that pairs trading profitabi*lity has a monotonic relationship with the signficance level of cointegration; supercointegrated pairs are defined as those which reject the null hypothesis with a 1% signficance. This consists of a twp step selectrion procedure combining the method proposed by Johansen <cite data-cite="6461367/V8Z5VVI4"></cite> and <cite data-cite="6461367/XQLXTIIQ"></cite>. 

The implementation of the Johansen Cointegration test is shown below. [Relevant docs](https://www.statsmodels.org/stable/generated/statsmodels.tsa.vector_ar.vecm.coint_johansen.html) indicate the attributes of the object which is generated by the test. Further reading for interpreting the results of the trace and maximum eigenvalue tests may be found [here](https://en.wikipedia.org/wiki/Johansen_test). In particular, we are interested in strongly cointegrated pairs which pass both tests with a 99% confidence level, i.e. reject the null hypothesis that $\Pi$ does not have full rank with 99% certainty. 

In [4]:
import numpy as np
from statsmodels.tsa.vector_ar.vecm import coint_johansen

class JohansenResults:
    def __init__(self, timeseries, det_order=0, lagged_terms=14):
        timeseries = np.array(timeseries)
        try:
            result = coint_johansen(timeseries, det_order, lagged_terms)
        except np.linalg.LinAlgError as error:
            raise error
        self.rank_pi = np.linalg.matrix_rank(np.array(result.cvt))
        self.trace_stat = np.array(result.lr1)
        self.trace_stat_crit_vals = np.array(result.cvt)[:,2]
        self.max_eig_stat = np.array(result.lr2)
        self.max_eig_stat_crit_vals = np.array(result.cvm)[:,2]
        self.eigenvalues = np.array(result.eig)
        self.pi = np.array(result.evec)
        self.output = [
            self.trace_stat,
            self.trace_stat_crit_vals,
            self.max_eig_stat,
            self.max_eig_stat_crit_vals
        ]
        self.is_pair = None
        self.p_val = None
        self.pair_test()

    def pair_test(self):
        self.is_pair = None
        r_max = len(self.trace_stat) - 1
        stat = [
            self.trace_stat[r_max], 
            self.max_eig_stat[r_max]
            ]
        crit_val = [
            self.trace_stat_crit_vals[r_max], 
            self.max_eig_stat_crit_vals[r_max]
            ]
        test = [(stat[i] > crit_val[i]) for i in range(2)]

        if all(test):
            self.is_pair = 1
            self.p_val = np.max(stat)

The ```self.trace_stat``` and ```self.max_eigen_val``` are the $p$-values which ought to be compared to the third column of the ```crit_val``` variable for 99% confidence. The first and second columns of the  ```crit_vals``` are the 90% and 95% confidence levels respectively. This $p$-value is stored, as recommended by <cite data-cite="6461367/F7Z9Z9M5"></cite> to determine the relative "strength" of cointegrated pairs. The hedge ratio $\Pi$, which can also be thought of as the eigenvectors $\hat{\beta}$ with eigenvalues $\hat{\alpha}$ as the adjustment coefficients.

Now we need to run the Johansen test on each possible pair of companies in the S&P100. We start with the list of symbols we got in our webscrape and the test data we pulled from the IEX API.

In [5]:
from scripts.iex_timeseries import symbols, load_test_data
n_symbols = 100
fund_type = str(n_symbols)
symbol_list = symbols(fund_type)
iex_data = load_test_data(fund_type)

We want a list of dataframes that just contain the close values, as we will be running our cointegration tests on these timeseries only.

In [6]:
times = iex_data[symbol_list[0]].index
close_dfs = pd.DataFrame(index=times)
close_dfs.dropna(inplace=True)
for symb in symbol_list:
    df = iex_data[symb]
    try:
        close_dfs[symb] = iex_data[symb].close
    except KeyError:
        continue
    except AttributeError:
        continue
close_dfs.to_numpy().shape

(729, 100)

We can see that we have 100 timeseries with 729 rows of data, which is the number of observations we pulled from IEX. Now we are interested in the pairs which pass the Johansen cointegration test with a 99% confidence level. For each pair, other than the companies, we are interested in its eigenvectors (hedge ratio), eigenvalues, and *p*-value for supercointegration strength.

In [7]:
pairs_df = pd.DataFrame()
pairs = []
evecs = []
eigvals = []
pvals = []

Now we loop through all the companies. For each we run the Johansen test with all companies to the right of it in the list, forming a upper triangular matrix for visualisation in a heatmap (although ```plotly``` represents it as a lower triangular matrix).

In [8]:
johansen_pair_matrix = np.ones((n_symbols, n_symbols))
johansen_pair_matrix[:] = None

for i in range(n_symbols):
    symbol = symbol_list[i]
    for j in range(i+1, n_symbols):
        next_symbol = symbol_list[j]

        try:
            ts = close_dfs[[symbol, next_symbol]].to_numpy()
        except KeyError:
            pass

        try:
            jh = JohansenResults(ts)
        except np.linalg.LinAlgError:
            continue

        johansen_pair_matrix[i, j] = jh.is_pair

        if jh.is_pair is not None:
            pairs.append((symbol, next_symbol))
            evecs.append(jh.pi)
            eigvals.append(jh.eigenvalues)
            pvals.append(jh.p_val)

We want to represent this in a dataframe and pickle it for later reference.

In [9]:
output_df = pd.DataFrame(index=range(len(pairs)))
output_df['Pair'] = pairs
output_df['Eigenvectors'] = evecs
output_df['Eigenvalues'] = eigvals
output_df['pValue'] = pvals
output_df.to_pickle('data/johansen_{}.pickle'.format(fund_type))

We can see relatively how many pairs occurred by representing our pair matrix as a heatmap and perform a quick check in case a certain stock had too many cointegrated pairs.

In [10]:
fig = go.Figure(data=go.Heatmap(
                    z=johansen_pair_matrix,
                    x=symbol_list,
                    y=symbol_list))
fig.update_layout(title_text='Johansen Cointegration Pair Results')
f1 = go.FigureWidget(fig)
f1

FigureWidget({
    'data': [{'type': 'heatmap',
              'uid': '8c6a7dd5-c158-4221-9449-f22b3c5541d4',
 …

It looks pretty reasonable. Drag on an area to zoom in and double click to zoom out again if you want to inspect an area of the map more closely.

In [11]:
df_jh = pd.read_pickle('data/johansen_100.pickle')
table_jh = ff.create_table(df_jh[['Pair', 'pValue']])
table1 = go.FigureWidget(table_jh)
table1

FigureWidget({
    'data': [{'colorscale': [[0, '#00083e'], [0.5, '#ededee'], [1, '#ffffff']],
              '…

In [12]:
len(df_jh)

33

So we have 33 pairs by the Johansen Cointegration test. This seems to approximately match the literature as in <cite data-cite="6461367/F7Z9Z9M5"></cite>. Now we need to conduct the Augmented Dickey-Fuller test to refine our cointegration pairs, which is quite straight forward to test for positive correlation coefficients in an ordinary least squares (OLS) linear regression.

### Augmented Dicky-Fuller Test Implementation

These pairs are further refined by the Engle & Granger method, also known as the Cointegrated Augmented Dicky Fuller Test (CADF), consisting of a refinement by OLS regression by pairs when both $\hat{\alpha}$ and $\hat{\beta}$ estimates are positive <cite data-cite="6461367/F7Z9Z9M5"></cite> and ordering them by their p-value.

Here are two helper functions. The first creates a dataframe from the two symbols in the pair based on their close values as the independent variable for interpolation. It then assigns the exogenous variables $\alpha$ and $\beta$ 

In [13]:
import statsmodels.api as sm

def df_for_ols(pair):
    symbol1, symbol2 = pair
    df = pd.DataFrame()
    df['y'] = iex_data[symbol1].close.to_list()
    df['beta'] = iex_data[symbol2].close.to_list()
    df['alpha'] = 1
    return df    

def ADFTest(pair):
    subdf = df_for_ols(pair)
    regression = sm.OLS(
        endog=subdf['y'], 
        exog=subdf[['alpha', 'beta']], 
        missing='drop'
    )
    results = regression.fit()
    alpha_pos, beta_pos = [(param > 0) for param in results.params]
    return (alpha_pos and beta_pos)

Now the ```ADFTest``` function will run an OLS regression between $y_t$ and $x_t$, the timeseries for symbols 1 and 2 respectively. I've tried to make the syntax look somewhat like the terminology in the paper which we have followed faithfully thus far. 

$$
    y_t = \alpha + \beta x_t + z_t
$$

Where $z_t$ is a normally distributed error term. The exogenous variables in the ```sm.OLS``` method call actually scales coefficients on these exogenous columns provided by the dataframe to perform a linear regression with an intercept coefficient $\alpha$. Otherwise, without adding the ```df['alpha']=1``` column would result in an OLS with a zero intercept. The docs for this function can be found [here](https://www.statsmodels.org/dev/examples/notebooks/generated/ols.html).

In [14]:
df_jh['ADFPassed'] = [ADFTest(pair) for pair in pairs]
adf_fn = 'data/post_adf_100.pickle'
df_jh.to_pickle(adf_fn)

In [15]:
post_adf_df = pd.read_pickle('data/post_adf_100.pickle')
table_adf = ff.create_table(post_adf_df[['Pair', 'pValue','ADFPassed']])
table2 = go.FigureWidget(table_adf)
table2

FigureWidget({
    'data': [{'colorscale': [[0, '#00083e'], [0.5, '#ededee'], [1, '#ffffff']],
              '…

Which finally leaves us with our nice 28 supercointegrated pairs with their hedge ratio found (eigenvector and eigenvalues) and ordered by their predicted profitability (p-value). The ADFPassed column can now be dropped as it no longer holds any useful information.

In [16]:
supercointegrated_df = post_adf_df.loc[post_adf_df['ADFPassed']]
supercointegrated_df.index = range(len(supercointegrated_df))
supercointegrated_df.drop(columns='ADFPassed', inplace=True)
len(supercointegrated_df)

28

Now let's save all the unique labels to its own .csv file so rerunning the code to find this won't be necessary, and I want to upload this dataset to Kaggle to use their free GPU architecture, which would be neater as a csv file rather than pulling from our ```pickle``` object. Declare an empty list and fill it with ticker symbols if the symbol does not already exist in the list.

In [17]:
interesting_symbols = []
for pair in supercointegrated_df.Pair.to_list():
    for symbol in pair:
        if not symbol in interesting_symbols:
            interesting_symbols.append(symbol)
interesting_symbols.sort() # To be in alphabetical order again, we have another datastructure for "strength" of cointegration
pd.DataFrame(columns=interesting_symbols)

,ALL,BIIB,BKNG,C,CL,COF,CVX,DUK,FB,GD,...,IBM,JNJ,KMI,MET,NVDA,QCOM,SPG,T,UPS,USB


Looks good. The df above was just to easily and prettily display the list of interesting symbols. Now pull these symbols from our pickled dataframe we prepared earlier and using a dict comprehension to extract only the symbols of interest to a new dictionary. 

In [18]:
working_data = dict((k, iex_data[k]) for k in interesting_symbols 
                                        if k in iex_data) 

Now we check that we only have the symbols of interest in our new working dictionary by using the dictionary unpacking operator *.

In [19]:
[*working_data] == interesting_symbols

True

Now we have our inputs for our NN. We should get into some trading theory to define our outputs.

## Trading Criteria

### Dynamic volatility threshold 

This is a threshold based on spread of the supercointegrated pair. In the case of our first pair, (BKNG: Booking Holdings Inc., UPS: United Parcel Service, Inc.), we can plot the spread with a 3-month rolling window of volatility with $\mu_w$ and $\sigma_w$ being the half-year mean and standard deviation respectively. For example, for our first pair we'll need the spread.

In [20]:
pair = supercointegrated_df.Pair.iloc[5]
df1, df2 = [working_data[symbol] for symbol in pair]
df_combined = pd.DataFrame(index=df1.index)
df_combined['spread'] = df1.close - df2.close
# Create traces
fig = go.Figure()

fig.add_trace(go.Scatter(x=df_combined.index, 
                         y=df_combined.spread, 
                         name="Spread",
                         line_color='red'))


title = 'Spread for {} - {}'.format(*pair)
valid_df = df_combined.dropna()
valid_timestamps = valid_df.index.to_list()
first_entry, last_entry = [valid_timestamps[0], valid_timestamps[-1]]
fig.update_layout(title_text=title,xaxis_range=[first_entry, last_entry],xaxis_rangeslider_visible=True)
fig2 = go.FigureWidget(fig)
fig2

FigureWidget({
    'data': [{'line': {'color': 'red'},
              'name': 'Spread',
              'type': '…

We can now write a rolling standard deviation (volatility) and rolling mean of 63 days (3 months).

In [21]:
rolling = df_combined.spread.rolling(63)
df_combined['mu_w'] = rolling.mean()
df_combined['sigma_w'] = rolling.std()
df_combined['lower_threshold'] = df_combined.mu_w - 2 * df_combined.sigma_w
df_combined['upper_threshold'] = df_combined.mu_w + 2 * df_combined.sigma_w
df_combined.tail()

,spread,mu_w,sigma_w,lower_threshold,upper_threshold
date,,,,,
2019-11-18,1843.44,1987.320476,66.544510,1854.231456,2120.409497
2019-11-19,1840.91,1984.623333,68.975415,1846.672504,2122.574163
2019-11-20,1825.08,1981.996349,71.837127,1838.322095,2125.670603
2019-11-21,1861.23,1981.886508,72.019069,1837.848369,2125.924647
2019-11-22,1843.62,1981.287460,73.018746,1835.249968,2127.324953


We can plot these easily to start visualising our trading strategy.

In [22]:
# Create traces
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_combined.index, 
                         y=df_combined.lower_threshold, 
                         name="Lower Threshold",
                         line_color='deepskyblue'))

fig.add_trace(go.Scatter(x=df_combined.index, 
                         y=df_combined.spread, 
                         name="Spread",
                         line_color='red'))

fig.add_trace(go.Scatter(x=df_combined.index, 
                         y=df_combined.mu_w, 
                         name="Rolling mean",
                         line_color='green'))

fig.add_trace(go.Scatter(x=df_combined.index, 
                         y=df_combined.upper_threshold, 
                         name="Upper Threshold",
                         line_color='dimgray'))

title = 'Spread with conditional volaility thresholds for {} - {}'.format(*pair)
valid_df = df_combined.dropna()
valid_timestamps = valid_df.index.to_list()
first_entry, last_entry = [valid_timestamps[0], valid_timestamps[-1]]
fig.update_layout(title_text=title,xaxis_range=[first_entry, last_entry],xaxis_rangeslider_visible=True)
fig3 = go.FigureWidget(fig)
fig3

FigureWidget({
    'data': [{'line': {'color': 'deepskyblue'},
              'name': 'Lower Threshold',
      …

So now we get to see why pairs trading is so popular. Feel free to move the slider around to focus in on a more granular set of timestamps. You can see above that when the spread reaches one of the thresholds it soon reverts toward the rolling mean again. Also it seems that our choice of a 3-month rolling window was a judicious choice. The max lag time in the Johansen Cointegration test was chosen at 2 weeks, which is a sampling rate more than twice the frequency of a 3-month window.

Given that we see the mean-reverting behaviour of the above timeseries, the essence of a pairs trading strategy would be to short the overperforming stock and long the underperforming stock on threshold crossing. In the case of an upper threshold crossing would indicate that by this pair, ALL was overperforming and COF was underperforming. Therefore short ALL and long COF. But in what degree? This is where our handy hedge ratio comes into play. The Johansen eigenvectors. 

I will go into this in the next article as an introduction to the NN that will be part of this model.

### References
<div class="cite2c-biblio"></div>